In [ ]:
### ADD LOCAL TO SMET FILES IN SUBFOLDERS ###
### TO MATCH PAUL SUBGRIDDING ###

from pathlib import Path

# Main folder
folder = Path("/home/benjamin/Documents/Doc/SHorton_HRDPS/nwp_archives/output/")

# Recursively find all .smet files in subfolders
files_found = list(folder.rglob("*.smet"))
print(f"Found {len(files_found)} .smet files.")

for file_path in files_found:
    # Skip files that already have "_local"
    if "_local" in file_path.stem:
        print(f"Skipping {file_path.name}, already has _local")
        continue

    # Build new name
    new_name = file_path.stem + "local" + file_path.suffix
    new_path = file_path.with_name(new_name)

    # Safety check: do not overwrite existing file
    if new_path.exists():
        print(f"Skipping {file_path.name}, target {new_name} already exists")
        continue

    # Rename
    file_path.rename(new_path)
    print(f"Renamed {file_path.name} -> {new_name}")

Found 1083 .smet files.
Renamed GNP_HRDPS_OLD_177.smet -> GNP_HRDPS_OLD_177local.smet
Renamed GNP_HRDPS_OLD_292.smet -> GNP_HRDPS_OLD_292local.smet
Renamed GNP_HRDPS_OLD_315.smet -> GNP_HRDPS_OLD_315local.smet
Renamed GNP_HRDPS_OLD_142.smet -> GNP_HRDPS_OLD_142local.smet
Renamed GNP_HRDPS_OLD_50.smet -> GNP_HRDPS_OLD_50local.smet
Renamed GNP_HRDPS_OLD_110.smet -> GNP_HRDPS_OLD_110local.smet
Renamed GNP_HRDPS_OLD_348.smet -> GNP_HRDPS_OLD_348local.smet
Renamed GNP_HRDPS_OLD_136.smet -> GNP_HRDPS_OLD_136local.smet
Renamed GNP_HRDPS_OLD_56.smet -> GNP_HRDPS_OLD_56local.smet
Renamed GNP_HRDPS_OLD_146.smet -> GNP_HRDPS_OLD_146local.smet
Renamed GNP_HRDPS_OLD_210.smet -> GNP_HRDPS_OLD_210local.smet
Renamed GNP_HRDPS_OLD_111.smet -> GNP_HRDPS_OLD_111local.smet
Renamed GNP_HRDPS_OLD_99.smet -> GNP_HRDPS_OLD_99local.smet
Renamed GNP_HRDPS_OLD_114.smet -> GNP_HRDPS_OLD_114local.smet
Renamed GNP_HRDPS_OLD_309.smet -> GNP_HRDPS_OLD_309local.smet
Renamed GNP_HRDPS_OLD_34.smet -> GNP_HRDPS_OLD_34loc

In [1]:
from pathlib import Path
import pyproj

# Folder with all SMET files
folder = Path("/home/benjamin/Documents/Doc/SHorton_HRDPS/nwp_archives/output/")

# EPSG conversion
wgs84 = pyproj.CRS("EPSG:4326")  # lon/lat
utm11 = pyproj.CRS("EPSG:32611") # UTM
transformer = pyproj.Transformer.from_crs(wgs84, utm11, always_xy=True)

# Loop over all SMET files recursively
for file_path in folder.rglob("*.smet"):
    with open(file_path, "r") as f:
        lines = f.readlines()

    # --- Extract existing info ---
    header_info = {}
    for line in lines:
        if "=" in line:
            key, value = line.split("=", 1)
            header_info[key.strip().lower().replace(" ", "_")] = value.strip()

    # Default values if missing
    station_id = header_info.get("station_id", file_path.stem)
    lat = float(header_info.get("latitude", 0))
    lon = float(header_info.get("longitude", 0))
    alt = header_info.get("altitude", header_info.get("altitude", "0"))
    nodata = header_info.get("nodata", "-999")

    # Convert lat/lon to UTM 32611
    easting, northing = transformer.transform(lon, lat)

    # Build new header in correct order
    new_header = [
        "SMET 1.1 ASCII\n",
        "[HEADER]\n",
        f"station id = {station_id}\n",
        f"latitude = {lat:.6f}\n",
        f"longitude = {lon:.6f}\n",
        f"easting = {easting:.2f}\n",
        f"northing = {northing:.2f}\n",
        "epsg = 32611\n",
        f"nodata = {nodata}\n",
        "fields = time_str TA RH VW WD ISWR ILWR PSUM\n",
        f"station name = {station_id}\n",
        f"altitude = {alt}\n",
        "tz = 0\n",
        "slope angle = 0\n",
        "slope azi = 0\n",
        "source = Simon Horton HRDPS backup from FIR compute canada\n",
        "\n"
    ]

    # --- Replace old header with new header ---
    # Find where header ends (usually after [HEADER] block)
    data_start_idx = 0
    for i, line in enumerate(lines):
        if line.strip() == "[HEADER]":
            # Data starts after the next blank line or after header keys
            data_start_idx = i + len(header_info) + 1
            break

    # Keep any data lines (observations) intact
    data_lines = lines[data_start_idx:]
    new_lines = new_header + data_lines

    # Write back to the same file
    with open(file_path, "w") as f:
        f.writelines(new_lines)

    print(f"Updated header for {file_path.name}")


Updated header for GNP_HRDPS_OLD_172local.smet
Updated header for GNP_HRDPS_OLD_120local.smet
Updated header for GNP_HRDPS_OLD_26local.smet
Updated header for GNP_HRDPS_OLD_233local.smet
Updated header for GNP_HRDPS_OLD_326local.smet
Updated header for GNP_HRDPS_OLD_77local.smet
Updated header for GNP_HRDPS_OLD_253local.smet
Updated header for GNP_HRDPS_OLD_352local.smet
Updated header for GNP_HRDPS_OLD_353local.smet
Updated header for GNP_HRDPS_OLD_8local.smet
Updated header for GNP_HRDPS_OLD_177local.smet
Updated header for GNP_HRDPS_OLD_235local.smet
Updated header for GNP_HRDPS_OLD_210local.smet
Updated header for GNP_HRDPS_OLD_25local.smet
Updated header for GNP_HRDPS_OLD_341local.smet
Updated header for GNP_HRDPS_OLD_71local.smet
Updated header for GNP_HRDPS_OLD_66local.smet
Updated header for GNP_HRDPS_OLD_332local.smet
Updated header for GNP_HRDPS_OLD_308local.smet
Updated header for GNP_HRDPS_OLD_30local.smet
Updated header for GNP_HRDPS_OLD_199local.smet
Updated header for GN